In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../pax"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import waveform_utils

from pax import core

pp = pprint.PrettyPrinter(depth=6)



processor MainProcess L66 INFO This is PAX version 6.9.0, running with configuration for XENON1T.


You don't have ROOT or root_pandas, root truth file output is disabled


WriteZipped MainProcess L223 INFO Overwriting output directory dummy_waveforms_1t


In [2]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(20, 10))

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    sum_wf = None
    event  = None
    df_zip_merged = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

      
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        #if (nInteractions != 1): continue

            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
     
        ############################################################################################
        ############################################################################################
    
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        continue
      
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged

    

In [3]:
import glob

####################################################################################################
####################################################################################################

#ver = 'test'
ver = datetime.datetime.now().strftime("%y%m%d%H%M")
ver = 'test'

nEventsPerFile = 2 #1000
nFilesZip      = 1 #200
nEvents        = nEventsPerFile * nFilesZip
dir_input    = '../pax_run/pax_output/Mar25/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
file_pkl     = 'merged/merged_pax_' + str(nEvents % 1000) + 'k_' + ver + '.pkl'
lst_contents = glob.glob(dir_format)

if (nFilesZip == -1):
    
    nFilesZip    = len(lst_contents)

print()
print("PAX input files: " + str(nFilesZip))
print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged = processPklEvents(zipfilename)
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

print(file_pkl)

df.to_pickle(file_pkl)



Done
merged/merged_pax_2k_test.pkl
